In [ ]:


import tensorflow as tf
import numpy as np
import random
import re
import os

with open('los3.txt', 'r') as f:
    text = f.read()

# Convertir el texto en minúsculas
text = text.lower()

# Eliminar caracteres especiales y números
text = re.sub(r'[^a-zñáéíóúü]', ' ', text)

# Crear un diccionario de caracteres únicos y asignar un número a cada uno
char_to_idx = {char: idx for idx, char in enumerate(sorted(set(text)))}

# Crear el diccionario inverso
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Convertir el texto a una lista de números
text_as_int = [char_to_idx[char] for char in text]

# Longitud de secuencia para la entrada del modelo
seq_length = 32

# Número de ejemplos de entrenamiento por lote
batch_size = 64

# Tamaño del vocabulario
vocab_size = len(char_to_idx)

# Número de unidades en la capa GRU
rnn_units = 512

# Crear una matriz de entrada y salida para el modelo
def create_dataset(text_as_int):
    inputs = []
    outputs = []

    for i in range(len(text_as_int) - seq_length):
        inputs.append(np.expand_dims(text_as_int[i:i+seq_length], axis=1))
        outputs.append(text_as_int[i+seq_length])

    return np.array(inputs), np.array(outputs)

# Crear el conjunto de datos de entrenamiento
inputs, outputs = create_dataset(text_as_int)

vocab_size

rnn_units

# Crear el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, rnn_units),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.GRU(rnn_units),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# Función de pérdida
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compilar el modelo
model.compile(optimizer='adam', loss=loss,metrics=['accuracy'])

# Crear un punto de control para guardar el modelo
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


# Entrenar el modelo
history = model.fit(inputs, outputs, batch_size=batch_size, epochs=10, callbacks=[checkpoint_callback])

# Guardar el modelo en un archivo hdf5
model.save('modfi.h5')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


49/49 [==============================] - 61s 1s/step - loss: 2.8635 - accuracy: 0.2082
Epoch 2/10
49/49 [==============================] - 55s 1s/step - loss: 2.2961 - accuracy: 0.3263
Epoch 3/10
49/49 [==============================] - 53s 1s/step - loss: 1.9144 - accuracy: 0.4154
Epoch 4/10
49/49 [==============================] - 53s 1s/step - loss: 1.6767 - accuracy: 0.4884
Epoch 5/10
49/49 [==============================] - 53s 1s/step - loss: 1.4752 - accuracy: 0.5512
Epoch 6/10
49/49 [==============================] - 53s 1s/step - loss: 1.2568 - accuracy: 0.6001
Epoch 7/10
49/49 [==============================] - 53s 1s/step - loss: 1.0604 - accuracy: 0.6651
Epoch 8/10
49/49 [==============================] - 52s 1s/step - loss: 0.8043 - accuracy: 0.7423
Epoch 9/10
49/49 [==============================] - 55s 1s/step - loss: 0.5925 - accuracy: 0.8105
Epoch 10/10
16/49 [========>.....................] - ETA: 35s - loss: 0.4246 - accuracy: 0.8770

#Generar Texto

In [ ]:
def generate_text(model, start_string, num_generate=10, temperature=1.0):
    string = start_string.lower()
    print(string)
    string = re.sub(r'[^a-zñáéíóúü]', ' ', string)
    input_eval = [char_to_idx[char] for char in string]
    if len(input_eval) < seq_length:
        input_eval = tf.pad(input_eval, [(0, seq_length - len(input_eval))], mode='CONSTANT', constant_values=0)
    input_eval = tf.reshape(input_eval, (1, seq_length))

    # Crear una lista vacía para almacenar el texto generado
    text_generated = []

    # Establecer la temperatura
    model.reset_states()
    temperature = temperature

    for i in range(num_generate):
        # Predecir el siguiente carácter utilizando el modelo
        predictions = model(input_eval)
       
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
       
        # Añadir el carácter generado a la lista de texto generado
        text_generated.append(idx_to_char[predicted_id])
        

        # Utilizar el carácter generado como entrada para la siguiente predicción
        input_eval = tf.concat([input_eval[:, 1:], tf.expand_dims([predicted_id], 1)], axis=-1)



    return ( ''.join(text_generated))


In [ ]:
# Generar texto nuevo
generated_text = generate_text(model, start_string='cerditos', num_generate=10, temperature=0.5)
print()
# Imprimir el texto generado
print(generated_text)